**Run on TPU - requires high RAM**

In [ ]:
!sudo apt-get update
!sudo apt-get install -y curl apt-transport-https ssl-cert ca-certificates gnupg lsb-release
!curl -1sLf 'https://dl.cloudsmith.io/public/wand/libwandio/cfg/setup/bash.deb.sh' | sudo -E bash
!echo "deb https://pkg.caida.org/os/$(lsb_release -si|awk '{print tolower($0)}') $(lsb_release -sc) main" | sudo tee /etc/apt/sources.list.d/caida.list
!sudo wget -O /etc/apt/trusted.gpg.d/caida.gpg https://pkg.caida.org/os/ubuntu/keyring.gpg
!sudo apt update; sudo apt-get install bgpstream

In [ ]:
!pip install pybgpstream
!python3 -m pip install pybgpkit-parser
!python3 -m pip install pybgpkit
!pip install neo4j
!pip install pycountry

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/LLM4BGP

Mounted at /content/drive
/content/drive/MyDrive/LLM4BGP


In [20]:
import pybgpstream
import networkx as nx
from itertools import groupby
import time
import json
import random
import pickle
from datasets.bgpstream_preparation import *

**Graph Problems**

In [5]:
def load_graph_from_pickle(pickle_filename):
    """Load a NetworkX graph from a pickle file"""
    with open(pickle_filename, 'rb') as f:
        graph = pickle.load(f)
    return graph

In [6]:
# Find the shortest path between two ASes given the AS graph
def shortest_path(as1, as2):
    as_graph = load_graph_from_pickle('datasets/routes/as_graph.pkl')
    path = nx.shortest_path(as_graph, str(as1), str(as2))
    path_len = len(path) - 1
    return path, path_len

In [7]:
shortest_path('43381', '213331')

(['43381', '19551', '3257', '2611', '213331'], 4)

In [8]:
# Check if a certain path is the shortest path between two ASNs
def is_shortest_path(source, target, path):
    """Check if a given path is the shortest path between source and target in graph G."""
    as_graph = load_graph_from_pickle('datasets/routes/as_graph.pkl')
    try:
        shortest_length = nx.shortest_path_length(as_graph, source, target)
        return len(path) - 1 == shortest_length
    except nx.NetworkXNoPath:
        return False

In [9]:
is_shortest_path('43381', '213331', ['43381', '19551', '3257', '2611', '213331'])

True

In [10]:
# Find all paths with exact length in the AS graph
def paths_with_exact_length(source, target, length):
    """
    Returns all simple paths in an undirected graph G from source to target
    that have exactly `length` edges.
    """
    as_graph = load_graph_from_pickle('datasets/routes/as_graph.pkl')
    all_paths = nx.all_simple_paths(as_graph, source, target, cutoff=length)
    return [path for path in all_paths if len(path) - 1 == length]

In [11]:
# paths_with_exact_length('43381', '213331', 4)

In [12]:
def as_degree_from_as_graph(asn):
    '''
    Returns the degree of an ASN from BGPStream indirected AS graph.
    Input: asn (int | str)
    Output: degree (int)
    '''
    as_graph = load_graph_from_pickle('datasets/routes/as_graph.pkl')
    return as_graph.degree(str(asn))

In [13]:
as_degree_from_as_graph('1299')

2423

**AS Paths Problems**

In [ ]:
def get_random_routes(x, asn):
    with open("datasets/routes/routes_by_asn.json", "r", encoding="utf-8") as f:
        routes_by_asn = json.load(f)   # dict
    asn_routes = routes_by_asn[str(asn)]
    if len(asn_routes) < x:
        x = asn_routes
    return random.sample(asn_routes, k=x)